In [1]:
from coir.data_loader import get_tasks
from coir.evaluation import COIR
from coir.models import YourCustomDEModel
import pandas as pd

/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/envs/virtual-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tasks = get_tasks(tasks=["cosqa"])

20604it [00:00, 208271.23it/s]


In [3]:
tasks['cosqa'][0]['d1']

{'text': 'def writeBoolean(self, n):\n        """\n        Writes a Boolean to the stream.\n        """\n        t = TYPE_BOOL_TRUE\n\n        if n is False:\n            t = TYPE_BOOL_FALSE\n\n        self.stream.write(t)',
 'title': ''}

In [4]:
expl_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CoSQA_deepseek_exp.csv'
expl_df = pd.read_csv(expl_path)
expl_df.head()

,query-id,corpus-id,score,partition_x,code,title_x,language_x,meta_information_x,partition_y,doc,title_y,language_y,meta_information_y,deepseek,explanation_deepseek
0,q1,d1,0,train,"def writeBoolean(self, n):\n """"""\n ...",NaN,PYTHON,{'dummy_field': ''},train,python code to write bool value 1,NaN,NaN,{'dummy_field': ''},NaN,The doc and code are meant to write a boolean ...
1,q2,d2,0,train,"def paste(xsel=False):\n """"""Returns system ...",NaN,PYTHON,{'dummy_field': ''},train,"""python how to manipulate clipboard""",NaN,NaN,{'dummy_field': ''},NaN,</think>\n\nThe function `paste()` pastes the ...
2,q3,d3,0,train,"def _format_json(data, theme):\n """"""Pretty ...",NaN,PYTHON,{'dummy_field': ''},train,python colored output to html,NaN,NaN,{'dummy_field': ''},NaN,</think>\n\nThe function `_format_json` takes ...
3,q4,d4,0,train,"def create_path(path):\n """"""Creates a absol...",NaN,PYTHON,{'dummy_field': ''},train,"python ""create directory"" using ""relative path""",NaN,NaN,{'dummy_field': ''},NaN,</think>\n\nThe code snippet creates an absolu...
4,q5,d5,0,train,"def _vector_or_scalar(x, type='row'):\n """"""...",NaN,PYTHON,{'dummy_field': ''},train,python column of an array,NaN,NaN,{'dummy_field': ''},NaN,</think>\n\nThe code converts an object into a...


In [5]:
# # Unpack the tuple
corpus, queries, qrels = tasks['cosqa']

for _, row in expl_df.iterrows():
    corpus_id = row['corpus-id']
    explanation = row['explanation_deepseek']

    if corpus_id in corpus and explanation and explanation.strip():
        corpus[corpus_id]['text'] = explanation  

corpus = {doc_id: doc for doc_id, doc in corpus.items() if 'text' in doc and doc['text'].strip()}
print(corpus['d1'])
tasks['cosqa'] = (corpus, queries, qrels)

print(f"Total docs in corpus after replacement: {len(tasks['cosqa'][0])}")
missing_text = sum(1 for doc in tasks['cosqa'][0].values() if not doc.get('text'))
print(f"Number of docs still missing 'text': {missing_text}")


{'text': 'The doc and code are meant to write a boolean value 1 to the stream.\n</think>\n\nThe doc and code are meant to write a boolean value 1 to the stream.\n\n```explanation```', 'title': ''}
Total docs in corpus after replacement: 20604
Number of docs still missing 'text': 0


In [6]:
model_name = "Linq-AI-Research/Linq-Embed-Mistral"
model = YourCustomDEModel(model_name=model_name)

Loading checkpoint shards: 100%|██████████| 3/3 [00:39<00:00, 13.24s/it]


In [7]:
evaluation = COIR(tasks=tasks,batch_size=128)

In [8]:
results = evaluation.run(model, output_folder=f"results/{model_name}")
print(results)

in evaluation.py: loading up dres

in evaluation.py: loading up retriever

in evaluation.py: retrieving

in beir/retrieval/evaluation.py: loading up search

in exact_search.py


Encoding batches:   0%|          | 0/161 [00:01<?, ?batch/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 4.00 GiB (GPU 0; 44.52 GiB total capacity; 36.62 GiB already allocated; 2.86 GiB free; 41.16 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF